## This notebook includes the code used in Algorithm 1 explained in the EEE4022S Traffic Monitoring Using Doppler Radar project report

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import spectrogram
from matplotlib.ticker import FuncFormatter
from scipy.stats import mode
import math

In [ ]:
# Load the WAV file
filename = ''  # Replace with your WAV file's path
sample_rate, audio_data = wavfile.read(filename)

# Create a spectrogram with adjusted parameters
frequencies, times, Sxx = spectrogram(audio_data, fs=sample_rate, nperseg=2048, noverlap=1024)

# Apply Gaussian smoothing
Sxx_smoothed = gaussian_filter(10 * np.log10(Sxx), sigma=(0.5, 3))

# Apply thresholding
threshold_db = -3  # Threshold in dB
Sxx_thresholded = np.where(Sxx_smoothed >= threshold_db, Sxx_smoothed, -80)

# Set all frequencies below 200 Hz and above 7000 Hz to -80 dB
lower_freq_limit = 700
upper_freq_limit = 7000

# Find the frequency indices corresponding to the limits
lower_freq_index = np.argmin(np.abs(frequencies - lower_freq_limit))
upper_freq_index = np.argmin(np.abs(frequencies - upper_freq_limit))

# Set the values outside the frequency limits to -80 dB
Sxx_thresholded[:lower_freq_index, :] = -80
Sxx_thresholded[upper_freq_index:, :] = -80

# Find the frequency index corresponding to the target frequency (750 Hz)
target_frequency = 750  # Hz
freq_index = np.argmin(np.abs(frequencies - target_frequency))

# Define the minimum group size and the power threshold
min_group_size = 3
power_threshold = 0  # dB

# Initialize variables to keep track of the group
group_start_time = None
group_size = 0

# Loop through the spectrogram at the specified frequency
for t in range(Sxx_thresholded.shape[1]):
    if Sxx_thresholded[freq_index, t] >= power_threshold:
        if group_start_time is None:
            group_start_time = t
            group_size = 1
        else:
            if t == group_start_time + group_size:
                group_size += 1
            else:
                if group_size >= min_group_size:
                    # Print the time range of the group
                    start_time = times[group_start_time]
                    end_time = times[group_start_time + group_size]
                    print(f"Group with {group_size} points >= {power_threshold} dB at times: {start_time} - {end_time}")

                    # Plot the full spectrogram for this group
                    group_start_index = group_start_time - 30
                    group_end_index = group_start_time + group_size
                    group_spectrogram = Sxx_thresholded[:, group_start_index:group_end_index]

                    # Find the indices of non-zero elements
                    nonzero_indices = np.argwhere(group_spectrogram != -80)

                    if len(nonzero_indices) > 0:
                        # Find the maximum indices for non-zero elements
                        max_indices = np.max(nonzero_indices, axis=0)
                        
                        #print("Largest indices of non-zero values:", tuple(max_indices))
                        print("max freq = ", frequencies[max_indices[0]])
                    else:
                        print("No non-zero values found in the array.")
                    
            

                    plt.figure(figsize=(12, 6))
                    plt.imshow(group_spectrogram, aspect='auto', cmap='inferno',
                               origin='lower', extent=[times[group_start_index], times[group_end_index], frequencies.min(), frequencies.max()])
                    plt.colorbar(label='dB')
                    plt.title(f'Spectrogram for Group (Threshold > {power_threshold} dB)')
                    plt.xlabel('Time (s)')
                    plt.ylabel('Frequency (Hz)')
                    plt.tight_layout()
                    plt.show()

                group_start_time = t
                group_size = 1

# Check if the last group meets the minimum size requirement
if group_size >= min_group_size:
    start_time = times[group_start_time]
    end_time = times[group_start_time + group_size]
    print(f"Group with {group_size} points >= {power_threshold} dB at times: {start_time} - {end_time}")

    # Plot the full spectrogram for the last group
    group_start_index = group_start_time - 25
    group_end_index = group_start_time + group_size
    group_spectrogram = Sxx_thresholded[:, group_start_index:group_end_index]

    # Find the indices of non-zero elements
    nonzero_indices = np.argwhere(group_spectrogram != -80)

    if len(nonzero_indices) > 0:
        # Find the maximum indices for non-zero elements
        max_indices = np.max(nonzero_indices, axis=0)
                        
        #print("Largest indices of non-zero values:", tuple(max_indices))
        print("max freq = ", frequencies[max_indices[0]])
    else:
        print("No non-zero values found in the array.")
    
    plt.figure(figsize=(12, 6))
    plt.imshow(group_spectrogram, aspect='auto', cmap='inferno',
               origin='lower', extent=[times[group_start_index], times[group_end_index], frequencies.min(), frequencies.max()])
    plt.colorbar(label='dB')
    plt.title(f'Spectrogram for Last Group (Threshold > {power_threshold} dB)')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    plt.show()